### This notebook deals with the instability arising out using the frs BC for traces and tries to find out if there is a mismatch between the input T and S and that at our boundaries at the time when the model is initialised (IC state is equal to the BC at time t = 0)

## The second part of this notebook also deals with the remaking of votemper and vosaline BC for NEMO (to remove the overturning effect being created in the model)

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import xarray as xr
import matplotlib.cm as cm
from scipy.interpolate import interp1d
from salishsea_tools import (nc_tools, gsw_calls, viz_tools)

/home/ssahu/anaconda3/lib/python3.5/site-packages/xarray/conventions.py:9: FutureWarning: The pandas.tslib module is deprecated and will be removed in a future version.
  from pandas.tslib import OutOfBoundsDatetime


#### Let us load the IC for our NEMO model

In [27]:
IC_file = nc.Dataset('/ocean/ssahu/CANYONS/wcvi/initial_conditions/West_coast_NEMO_IC_temperature_salinity_nomask_JP.nc')

In [28]:
votem_IC = IC_file.variables['votemper'][:];
vosal_IC = IC_file.variables['vosaline'][:];

In [29]:
votem_IC.shape

(1, 27, 100, 70)

#### Now let us load the T and S for the BC file (Let's begin the excersize by loading only the BC file of the west boundary)

In [1]:
BC_west = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/3d_NEMO_west_m04.nc');

NameError: name 'nc' is not defined

In [31]:
votem_west_BC = BC_west.variables['votemper'][:];
vosal_west_BC = BC_west.variables['vosaline'][:];

In [32]:
votem_west_BC.shape

(43, 27, 4, 100)

#### By principle the outer values at the very first time step should be equal to the value of the IC (let's test that)

In [33]:
BC_check = votem_west_BC[0,:,:,:]; print(BC_check.shape)

IC_check = votem_IC[0,:,:]; print (IC_check.shape)

(27, 4, 100)
(27, 100, 70)


In [34]:
np.mean(BC_check)

4.5696635

In [35]:
np.mean(IC_check[:,:,:4])

4.5006418

#### So they are not same and our fears just became real

In [11]:
BC_check[0,...]

array([[ 11.26034641,  11.24334621,  11.24070263,  11.23822403,
         11.22112751,  11.19831944,  11.20296288,  11.22289371,
         11.23872852,  11.26172924,  11.25253773,  11.18979836,
         11.12477112,  11.08434582,  11.03537273,  10.94834614,
         10.83931923,  10.76751041,  10.740798  ,  10.72731972,
         10.71774483,  10.70897865,  10.67700577,  10.63631916,
         10.61370277,  10.6175375 ,  10.62753773,  10.63834572,
         10.64879799,  10.65515423,  10.66360664,  10.67098999,
         10.66692066,  10.63546848,  10.56756401,  10.48611164,
         10.44856358,  10.44137287,  10.42972851,  10.41382504,
         10.40446854,  10.40392017,  10.41346836,  10.42592049,
         10.43372917,  10.43553734,  10.43325043,  10.42305851,
         10.41341496,  10.40431976,  10.40231895,  10.41522408,
         10.43938828,  10.46419716,  10.48419666,  10.5051012 ,
         10.52190971,  10.52310181,  10.50003242,  10.46222401,
         10.40486717,  10.32905865,  10.

In [25]:
np.where(IC_check[0,...] == np.max(IC_check[0,...]))

(array([0]), array([65]))

In [18]:
BC_check[0,0,0]

11.260346

In [24]:
np.where(IC_check[0,...] == BC_check[0,...])

/home/ssahu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  if __name__ == '__main__':


(array([], dtype=int64),)

In [23]:
np.where(BC_check[0,...] == np.max(BC_check[0,...]))

(array([3]), array([5]))

In [28]:
IC_check[0,0,60]

11.024442

In [29]:
BC_check[0,3,5]

11.303032

In [22]:
np.where(BC_check == IC_check)

/home/ssahu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  if __name__ == '__main__':


(array([], dtype=int64),)

## The temperature and salinity doesnot seem to work in the BC file and we have a overturning effect of salinity in the south BC file;

So we decide to make the BC files of temerature and salinity from the JP files straigh similar to the IC file but with a separate loop over time and and not from the numpy arrays (saved earlier)

In [2]:
import numpy as np

import netCDF4 as nc

import xarray as xr

from scipy.interpolate import griddata, interp1d

from salishsea_tools import nc_tools,viz_tools

import matplotlib.pyplot as plt

import matplotlib.cm as cm

from fancyimpute import KNN



In [3]:
fname = '/home/ssahu/saurav/JP_BC/cat_42_days_T.nc';

gridT = xr.open_dataset(fname);

In [4]:
gridT.keys

<bound method Mapping.keys of <xarray.Dataset>
Dimensions:            (bnds: 2, deptht: 50, time_counter: 43, x: 715, y: 1021)
Coordinates:
  * deptht             (deptht) float32 0.494025 1.54138 2.64567 3.81949 ...
  * time_counter       (time_counter) datetime64[ns] 2016-04-01T12:00:00 ...
  * y                  (y) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...
  * x                  (x) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...
  * bnds               (bnds) int64 0 1
Data variables:
    nav_lon            (y, x) float32 -142.283 -142.256 -142.228 -142.2 ...
    nav_lat            (y, x) float32 45.0491 45.0479 45.0467 45.0456 ...
    time_counter_bnds  (time_counter, bnds) float64 4.873e+07 4.881e+07 ...
    votemper           (time_counter, deptht, y, x) float64 0.0 8.881 8.905 ...
    vosaline           (time_counter, deptht, y, x) float64 0.0 33.14 33.13 ...
    sosstsst           (time_counter, y, x) float64 0.0 8.881 8.905 8.925 ...
    sosaline           (

In [5]:
#### Load JP's mask

mask = nc.Dataset('/data/mdunphy/NEP036-N30-OUT/INV/mesh_mask.nc');

tmask_JP = mask.variables['tmask'][0,:,55:400,446:701];
umask_JP = mask.variables['umask'][0,:,55:400,446:701];
vmask_JP = mask.variables['vmask'][0,:,55:400,446:701];

print(tmask_JP.shape, umask_JP.shape, vmask_JP.shape)

(50, 345, 255) (50, 345, 255) (50, 345, 255)


In [6]:
#### Slice out our domain of WCVI from JP and mask it at the same time saying the zero values as NAN

votemper_JP = np.array(gridT['votemper'][:,:,55:400,446:701]);
vosaline_JP = np.array(gridT['vosaline'][:,:,55:400,446:701]);


for i in np.arange(votemper_JP.shape[0]):
    votemper_JP[i,...] = np.ma.masked_array(votemper_JP[i,...], mask = tmask_JP[...]);
    vosaline_JP[i,...] = np.ma.masked_array(vosaline_JP[i,...], mask = tmask_JP[:,...]);

votemper_JP[votemper_JP == 0] =['Nan'];
vosaline_JP[vosaline_JP == 0] = ['Nan'];

glamt_bc_JP = np.array(gridT['nav_lon'][55:400,446:701]);
gphit_bc_JP = np.array(gridT['nav_lat'][55:400,446:701]);
deptht_JP = np.array(gridT['deptht'][:]);

In [7]:
votemper_JP.shape

(43, 50, 345, 255)

In [8]:
votemper_JP[0,:,0,0]

array([ 11.06409264,  10.9796772 ,  10.8708601 ,  10.7358036 ,
        10.56975365,  10.46552658,  10.39736271,  10.35089111,
        10.30435085,  10.2620573 ,  10.2235651 ,  10.19233513,
        10.16580296,  10.14525414,  10.12831116,  10.11257267,
        10.09665108,  10.08100414,  10.06139183,  10.02533722,
         9.9542942 ,   9.85076618,   9.47916126,   8.5789938 ,
         8.0148592 ,   7.63935375,   7.19570589,   6.54779291,
         5.85100937,   5.20056057,   4.81492567,   4.49857283,
         4.16982317,   3.88400245,   3.55892754,   3.20406389,
         2.82315612,   2.4409368 ,   2.08485484,   1.81899297,
         1.68972611,          nan,          nan,          nan,
                nan,          nan,          nan,          nan,
                nan,          nan])

In [13]:
#### Load the WCVI points

fname_wcvi = '/ocean/ssahu/CANYONS/wcvi/grid/coordinates.nc'

with nc.Dataset(fname_wcvi, 'r') as coord:
    gphit_wcvi = coord.variables['gphit'][0,...];
    glamt_wcvi =  coord.variables['glamt'][0,...]; 
    
print((glamt_bc_JP[0,0],gphit_bc_JP[0,0]), (glamt_wcvi[0,0],gphit_wcvi[0,0]))

(-129.93523, 45.455914) (-127.00188120221485, 46.113550613066906)


In [24]:
#Specify the rimwidth
    
N = 4

glamt_wcvi_bc_left = glamt_wcvi[:,:N]; gphit_wcvi_bc_left = gphit_wcvi[:,:N];
glamt_wcvi_bc_right = glamt_wcvi[:,-N:]; gphit_wcvi_bc_right = gphit_wcvi[:,-N:];

glamt_wcvi_top = glamt_wcvi[:N,:]; gphit_wcvi_top = gphit_wcvi[:N,:];
glamt_wcvi_bottom = glamt_wcvi[-N:,:]; gphit_wcvi_bottom = gphit_wcvi[-N:,:];


print(glamt_wcvi_bc_left.shape);
print(glamt_wcvi_bc_right.shape);
print(glamt_wcvi_bc_top.shape);
print(glamt_wcvi_bottom.shape);

(100, 4)
(100, 4)


NameError: name 'glamt_wcvi_bc_top' is not defined

In [22]:
#For the left boundary

X = glamt_bc_JP.flatten();

Y = gphit_bc_JP.flatten();

points = (X[:],Y[:]);

xi_left = (glamt_wcvi_bc_left.flatten(), gphit_wcvi_bc_left.flatten());

votemper_bc_left = np.zeros((votemper_JP.shape[0], votemper_JP.shape[1], glamt_wcvi_bc_left.shape[0], glamt_wcvi_bc_left.shape[1]));
vosaline_bc_left = np.zeros((vosaline_JP.shape[0], vosaline_JP.shape[1], glamt_wcvi_bc_left.shape[0], glamt_wcvi_bc_left.shape[1]));

for p in np.arange(votemper_JP.shape[0]):
    for i in np.arange(votemper_JP.shape[1]):
        votemper_bc_left[p,i,...] = np.reshape(griddata(points, votemper_JP[p,i,...].flatten(), xi_left, method= 'linear'), glamt_wcvi_bc_left.shape)
        vosaline_bc_left[p,i,...] = np.reshape(griddata(points, vosaline_JP[p,i,...].flatten(), xi_left, method= 'linear'), glamt_wcvi_bc_left.shape)




In [11]:
vosaline_bc_left[0,:,0,0]

array([ 32.47419656,  32.47339993,  32.472351  ,  32.47062415,
        32.46957545,  32.46912266,  32.46870302,  32.46790199,
        32.46697466,  32.46655073,  32.46634141,  32.4661859 ,
        32.4660519 ,  32.4658899 ,  32.46588335,  32.46623902,
        32.46850987,  32.47960524,  32.52179053,  32.63602269,
        32.85181797,  33.12211982,  33.38578935,  33.65309569,
        33.81709729,  33.93756474,  34.00150822,  34.03875474,
        34.0547015 ,  34.05655214,  34.07886744,  34.13459396,
        34.20362748,  34.26293978,  34.33264525,  34.39530034,
        34.4533169 ,  34.49530198,  34.5369522 ,  34.579813  ,
        34.62248934,  34.63830741,  34.64125478,          nan,
                nan,          nan,          nan,          nan,
                nan,          nan])

In [ ]:
for i in np.arange(vosaline_bc.shape[0]):
    for j in np.arange(vosaline_bc.shape[1]):
        votemper_ic[i,j,...] = KNN(k=3).complete(votemper_bc[i,j,...]);
        vosaline_ic[i,j,...] = KNN(k=3).complete(vosaline_bc[i,j,...]);

In [ ]:
vosaline_bc[0,:,0,0]

In [ ]:
vosaline_bc[np.where(np.isnan(vosaline_bc))]=0;
votemper_bc[np.where(np.isnan(votemper_bc))]=0;

for t in np.arange(votemper_bc.shape[0]):
    for i in np.arange(votemper_bc.shape[1]):
        for p in np.arange(votemper_bc.shape[2]):
            for l in np.arange(votemper_bc.shape[3]):
            if votemper_bc[t,i,p,l] == 0:
                votemper_bc[t,i,p,l] = votemper_bc[t,i-1,p,l]
            else:
                continue

for t in np.arange(vosaline_bc.shape[0]):
    for i in np.arange(vosaline_bc.shape[1]):
        for p in np.arange(vosaline_bc.shape[2]):
            for l in np.arange(vosaline_bc.shape[3]):
            if vosaline_bc[t,i,p,l] == 0:
                vosaline_bc[t,i,p,l] = vosaline_bc[t,i-1,p,l]
            else:
                continue


array([[-127.0018812 , -126.95601614, -126.91015107, -126.86428601],
       [-127.03306936, -126.9864822 , -126.93994715, -126.8934567 ],
       [-127.06419593, -127.01689616, -126.96969861, -126.92258885],
       [-127.09520221, -127.04720792, -126.99936229, -126.95164495],
       [-127.12603466, -127.07737102, -127.02889767, -126.98058953],
       [-127.15664632, -127.10734382, -127.05826784, -127.00938984],
       [-127.18699737, -127.13709008, -127.08744005, -127.03801637],
       [-127.21705497, -127.16657909, -127.1163859 , -127.06644312],
       [-127.24679273, -127.1957854 , -127.14508136, -127.09464765],
       [-127.27618994, -127.22468845, -127.17350647, -127.12261098],
       [-127.30523082, -127.25327197, -127.20164509, -127.15031743],
       [-127.3339037 , -127.28152354, -127.22948448, -127.17775435],
       [-127.36220044, -127.30943399, -127.25701496, -127.20491185],
       [-127.39011583, -127.33699703, -127.2842295 , -127.23178254],
       [-127.41764713, -127.364208

array([[-123.97478694, -123.92892187, -123.88305681, -123.83719174],
       [-123.97710275, -123.93103873, -123.88512943, -123.83947249],
       [-123.97931611, -123.93300086, -123.88694967, -123.8412887 ],
       [-123.98134543, -123.93469892, -123.88837971, -123.84251229],
       [-123.98313621, -123.9360697 , -123.88935793, -123.84311249],
       [-123.98465899, -123.93708573, -123.88986984, -123.84310827],
       [-123.98590449, -123.9377444 , -123.88992742, -123.84253689],
       [-123.98687877, -123.93805995, -123.88955855, -123.84144604],
       [-123.98759922, -123.93805809, -123.8888008 , -123.83988867],
       [-123.98809165, -123.9377724 , -123.88769788, -123.83792078],
       [-123.98838807, -123.93724196, -123.88629755, -123.83560008],
       [-123.98852517, -123.93650983, -123.88465026, -123.83298529],
       [-123.98854316, -123.93562194, -123.88280839, -123.83013571],
       [-123.988485  , -123.93462637, -123.88082564, -123.82711104],
       [-123.9883958 , -123.933572

#### Making the south boundary from the start

In [3]:
import numpy as np
import numpy.ma as ma
import netCDF4 as nc
import matplotlib.pyplot as plt
import matplotlib as mpl
from salishsea_tools import viz_tools, geo_tools,nc_tools
from scipy.interpolate import griddata, interp1d
import matplotlib.cm as cm
import xarray as xr
from grid_alignment import calculate_initial_compass_bearing as cibc

In [4]:
path = '/home/ssahu/saurav/JP_BC/'

In [5]:
gridT = xr.open_dataset(path+'cat_42_days_T.nc')

In [6]:
gridT.coords

Coordinates:
  * deptht        (deptht) float32 0.494025 1.54138 2.64567 3.81949 5.07822 ...
  * time_counter  (time_counter) datetime64[ns] 2016-04-01T12:00:00 ...
  * y             (y) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
  * x             (x) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
  * bnds          (bnds) int64 0 1

In [7]:
votemper_unfiltered = np.array(gridT['votemper'][:,:,55:400,446:701]);
vosaline_unfiltered = np.array(gridT['vosaline'][:,:,55:400,446:701]);
glamt_bc_JP = np.array(gridT['nav_lon'][55:400,446:701]);
gphit_bc_JP = np.array(gridT['nav_lat'][55:400,446:701]);

In [10]:
print(votemper_unfiltered.shape)

for a in np.arange(votemper_unfiltered.shape[0]):
    for i in np.arange(votemper_unfiltered.shape[1]):
        for l in np.arange(votemper_unfiltered.shape[2]):
            for m in np.arange(votemper_unfiltered.shape[3]):
                if votemper_unfiltered[a,i,l,m] == 0:
                    votemper_unfiltered[a,i,l,m] == ['Nan'];
                    
                    
for a in np.arange(vosaline_unfiltered.shape[0]):
    for i in np.arange(vosaline_unfiltered.shape[1]):
        for l in np.arange(vosaline_unfiltered.shape[2]):
            for m in np.arange(vosaline_unfiltered.shape[3]):
                if vosaline_unfiltered[a,i,l,m] == 0:
                    vosaline_unfiltered[a,i,l,m] == ['Nan'];
                

(43, 50, 345, 255)


/home/ssahu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/home/ssahu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [11]:
votemper = np.empty(votemper_unfiltered.shape);
vosaline = np.empty(vosaline_unfiltered.shape);
for idx, val in enumerate(votemper_unfiltered[:,...]):
    
    votemper[idx-1,...] = votemper_unfiltered[idx-2,...]*0.25 + votemper_unfiltered[idx-1,...]*0.5 + \
                                votemper_unfiltered[idx,...]*0.25;
    vosaline[idx-1,...] = vosaline_unfiltered[idx-2,...]*0.25 + vosaline_unfiltered[idx-1,...]*0.5 + \
                                vosaline_unfiltered[idx,...]*0.25;
votemper[0,...] = votemper_unfiltered[0,...];
vosaline[0,...] = vosaline_unfiltered[0,...];
        
print(votemper.shape, vosaline.shape)

(43, 50, 345, 255) (43, 50, 345, 255)


In [12]:
fname_wcvi = '/ocean/ssahu/CANYONS/wcvi/grid/coordinates.nc'


with nc.Dataset(fname_wcvi, 'r') as coord:
    gphit_wcvi = coord.variables['gphit'][0,...];
    glamt_wcvi =  coord.variables['glamt'][0,...];

In [13]:
X = glamt_bc_JP.flatten()

Y = gphit_bc_JP.flatten()

points = (X[:],Y[:])


#Number of points to trim off the ends

N = 5

glamt_wcvi_bc_bottom = glamt_wcvi[0:N-1,:]; gphit_wcvi_bc_bottom = gphit_wcvi[0:N-1,:];



xi_bottom = (glamt_wcvi_bc_bottom.flatten(), gphit_wcvi_bc_bottom.flatten());
#votemper_wcvi = np.zeros((43,50,Nx,Ny));

votemper_wcvi_bottom = np.empty((43,50,glamt_wcvi_bc_bottom.shape[0], glamt_wcvi_bc_bottom.shape[1]));

vosaline_wcvi_bottom = np.empty((43,50,glamt_wcvi_bc_bottom.shape[0], glamt_wcvi_bc_bottom.shape[1]));



for p in np.arange(votemper_wcvi_bottom.shape[0]):
    for i in np.arange(votemper_wcvi_bottom.shape[1]):
        votemper_wcvi_bottom[p,i,...] = np.reshape(griddata(points, votemper[p,i,...].flatten(), xi_bottom, method= 'linear'), glamt_wcvi_bc_bottom.shape)
        vosaline_wcvi_bottom[p,i,...] = np.reshape(griddata(points, vosaline[p,i,...].flatten(), xi_bottom, method= 'linear'), glamt_wcvi_bc_bottom.shape)


In [14]:
print(votemper_wcvi_bottomi_bottom.shape)

(43, 50, 4, 70)
